In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 34kB/s 
     |████████████████████████████████| 450kB 48.5MB/s 
     |████████████████████████████████| 3.8MB 37.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
import tensorflow
tensorflow.__version__

'2.1.0'

In [0]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/My Drive/Casia/Casia2/model_val_0.8391.h5",compile=True)

In [0]:
import numpy as np
import pywt


In [0]:
DATA_PATH = "/content/drive/My Drive/Casia/Casia2/"

# au_batches_train = np.load(DATA_PATH + "au_batches_train.npy")
# edge_batches_train = np.load(DATA_PATH + "edge_batches_train.npy")
# tp_batches_train = np.load(DATA_PATH + "tp_batches_train.npy")

In [0]:
X_au_test = np.load(DATA_PATH + "X_au_test.npy")
X_tp_test = np.load(DATA_PATH + "X_tp_test.npy")

In [0]:
def img2batch(arr,shape=(32,32)):

  nrows, ncols = shape
  batches = []
  dim = len(arr.shape)
  if dim == 3:
    h, w,_ = arr.shape
  elif dim ==2:
    h, w = arr.shape

  assert h % nrows == 0, "{} rows is not evenly divisble by {}".format(h, nrows)
  assert w % ncols == 0, "{} cols is not evenly divisble by {}".format(w, ncols)


  for r in range(h//nrows):
    for c in range(w//ncols):
      from_r = r*nrows
      to_r = (r+1)*nrows
      from_c = c*ncols
      to_c =(c+1)*ncols
      if dim == 3:
        batches.append(arr[from_r:to_r,from_c:to_c,:])
      elif dim==2:
        batches.append(arr[from_r:to_r,from_c:to_c])

  return batches
  
def mu_std_sum(matrix):
  return [np.mean(matrix),np.std(matrix),np.sum(matrix)]

def extract_features(image):

  # Transpose to iterate through channels
  image = np.transpose(image,(2,0,1))
  vector = []

  for channel in image[1:]: # Remove Y Channel

    # iterate through db1->db5
    for i in range(1,6):
      coeffs = pywt.wavedec2(channel,"db"+str(i),level=3)

      # Append first mat mu_std_sum
      vector += mu_std_sum(coeffs[0])

      for dec in coeffs[1:]:
        vector += mu_std_sum(dec[0])
        vector += mu_std_sum(dec[1])
        vector += mu_std_sum(dec[2])

  return vector
      


In [0]:
def img2batch(arr,shape=(32,32)):

  nrows, ncols = shape
  batches = []
  dim = len(arr.shape)
  if dim == 3:
    h, w,_ = arr.shape
  elif dim ==2:
    h, w = arr.shape

  assert h % nrows == 0, "{} rows is not evenly divisble by {}".format(h, nrows)
  assert w % ncols == 0, "{} cols is not evenly divisble by {}".format(w, ncols)


  for r in range(h//nrows):
    for c in range(w//ncols):
      from_r = r*nrows
      to_r = (r+1)*nrows
      from_c = c*ncols
      to_c =(c+1)*ncols
      if dim == 3:
        batches.append(arr[from_r:to_r,from_c:to_c,:])
      elif dim==2:
        batches.append(arr[from_r:to_r,from_c:to_c])

  return batches

In [0]:
def pkl_load(path):
  import pickle
  with open(path, 'rb') as fp:
    obj = pickle.load(fp)
  return obj

scaler_obj = pkl_load(DATA_PATH  + "scaler_obj.pkl")

In [0]:
import sklearn.preprocessing._data

In [0]:
from sklearn.externals import joblib
joblib.dump(scaler_obj, "scaler_obj.pkl") 


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['scaler_obj.pkl']

In [0]:
import pickle
pickle.__version__

AttributeError: ignored

In [0]:
import pickle 
print(pickle.format_version)

4.0


In [0]:
idx = 3
batches = img2batch(X_tp_test[idx])
label = []
for batch in batches:
  feature = np.array(extract_features(batch))
  feature = np.expand_dims(feature,0)
  feature = scaler_obj.transform(feature)
  y_hat = (model.predict(feature)[0][0]>0.5)*1
  label.append(y_hat)

In [0]:
idx = 4
def count_tampered(image):
  batches = img2batch(image)
  label = []
  for batch in batches:
    feature = np.array(extract_features(batch))
    feature = np.expand_dims(feature,0)
    feature = scaler_obj.transform(feature)
    y_hat = (model.predict(feature)[0][0]>0.5)*1
    label.append(y_hat)
  ones = collections.Counter(label)[1]
  return ones

In [0]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [0]:
from tqdm import tqdm_notebook

In [0]:
tp_pred = []
for item in tqdm_notebook(X_tp_test):
  tp_pred.append(count_tampered(item))

Streaming output truncated to the last 5000 lines.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will exp

In [0]:
au_pred = []
for item in tqdm_notebook(X_au_test):
  au_pred.append(count_tampered(item))

Streaming output truncated to the last 5000 lines.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will exp

In [0]:
def pkl_save(obj,path):
  import pickle
  with open(path, 'wb') as fp:
    pickle.dump(obj, fp)
pkl_save(au_pred,DATA_PATH+"au_pred.pkl")
pkl_save(tp_pred,DATA_PATH+"tp_pred.pkl")

In [0]:
au_count = pkl_load(DATA_PATH+"au_pred.pkl")
tp_count = pkl_load(DATA_PATH+"tp_pred.pkl")

In [0]:
thresh = 8
au_pred = (np.array(au_count)>=thresh)*1
tp_pred = (np.array(tp_count)>=thresh)*1
y_pred = np.concatenate((au_pred,tp_pred),axis=0)
y_true = np.concatenate((np.zeros(au_pred.shape[0]),np.ones(tp_pred.shape[0])))

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print("accuracy {0:.4f}".format(accuracy_score(y_true,y_pred)))
print("f_score {0:.4f}".format(f1_score(y_true,y_pred)))

accuracy 0.8618
f_score 0.8398
